In [ ]:
# 阶段2：宏观经济SubAgent实现

import sys
sys.path.insert(0, '../src')

# 加载环境变量
from dotenv import load_dotenv
load_dotenv('../config/.env')

# 使用封装好的create_macro_agent
from analyst_chain.agents import create_macro_agent

# 创建宏观经济分析Agent
agent = create_macro_agent()

print("[成功] 宏观经济SubAgent已创建")
print(f"   模型: DeepSeek")
print(f"   Tools: AKShare工具(3) + 知识检索(2)")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: /Users/Qunying/Project/llm/llm-lm-python/data/models/models--Qwen--Qwen3-Embedding-0.6B/snapshots/c54f2e6e80b2d7b7de06f51cec4959f6b3e03418
INFO:sentence_transformers.SentenceTransformer:1 prompt is loaded, with the key: query


✅ 宏观经济SubAgent已创建
   Tools数量: 5
   - AKShare工具: 3个 (GDP/CPI/PMI)
   - 知识检索: 2个 (向量检索/JSON查询)


## 测试场景

测试宏观经济SubAgent的综合分析能力


In [9]:
# 测试1：GDP分析
test_query_1 = "最近GDP增长率如何？有什么趋势特征？"
print(f"问题：{test_query_1}")
print("-" * 60)

# result = agent.invoke({"messages": [{"role": "user", "content": test_query_1}]})
# print(result["messages"][-1].content)


问题：最近GDP增长率如何？有什么趋势特征？
------------------------------------------------------------


In [10]:
# 测试2：通胀分析
test_query_2 = "当前通胀水平怎么样？根据经济理论应该如何解读？"
print(f"问题：{test_query_2}")
print("-" * 60)

# result = agent.invoke({"messages": [{"role": "user", "content": test_query_2}]})
# print(result["messages"][-1].content)


问题：当前通胀水平怎么样？根据经济理论应该如何解读？
------------------------------------------------------------


In [11]:
# 测试3：PMI经济趋势分析
test_query_3 = "PMI数据显示经济趋势如何？结合经济周期理论分析"
print(f"问题：{test_query_3}")
print("-" * 60)

# result = agent.invoke({"messages": [{"role": "user", "content": test_query_3}]})
# print(result["messages"][-1].content)


问题：PMI数据显示经济趋势如何？结合经济周期理论分析
------------------------------------------------------------


## 任务3：完整测试（10个场景）

测试SubAgent的综合分析能力，评估输出质量


In [ ]:
# 10个测试场景（按难度递增）
test_questions = [
    # 基础数据查询（简单）
    "2024年GDP增长率是多少？",
    "当前的通胀水平如何？",
    "最新的PMI数据是多少？",

    # 周期判断（中等）
    "当前经济处于什么周期？",
    "经济周期转折的信号是什么？",
    "什么指标变化会预示周期转折？",

    # 投资策略（困难）
    "根据当前经济周期，应该配置什么资产？",
    "投资时钟当前处于哪个阶段？",

    # 综合分析（最难）
    "给出当前宏观经济的整体判断",
    "从宏观角度看，周期性行业投资机会如何？"
]

print(f"共{len(test_questions)}个测试问题，按难度递增")
for i, q in enumerate(test_questions, 1):
    print(f"{i}. {q}")


共10个测试问题，按难度递增
1. 2024年GDP增长率是多少？
2. 当前的通胀水平如何？
3. 最新的PMI数据是多少？
4. 当前经济处于什么周期？
5. 经济周期转折的信号是什么？
6. 什么指标变化会预示周期转折？
7. 根据当前经济周期，应该配置什么资产？
8. 投资时钟当前处于哪个阶段？
9. 给出当前宏观经济的整体判断
10. 从宏观角度看，周期性行业投资机会如何？


In [ ]:
# 批量测试（注释状态，需要时取消下面的三引号运行）
"""
results = []

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*60}")
    print(f"测试 {i}/{len(test_questions)}: {question}")
    print('='*60)

    try:
        result = agent.invoke({"messages": [{"role": "user", "content": question}]})
        response_text = result["messages"][-1].content
        results.append({
            'question': question,
            'response': response_text,
            'status': 'success'
        })
        print(response_text)
    except Exception as e:
        results.append({
            'question': question,
            'error': str(e),
            'status': 'failed'
        })
        print(f"[错误] 错误: {e}")

# 保存测试结果
import json
with open('../data/stage2_test_results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n[完成] 测试完成！成功: {sum(1 for r in results if r['status']=='success')}/{len(results)}")
"""

print("测试脚本已准备，取消三引号可运行")


测试脚本已准备，取消三引号可运行


## 输出质量评估标准

**数据准确性**（30分）：
- [必须] 引用正确来源（"根据AKShare最新数据...）
- [必须] 数据时间明确（"2024年11月..."）
- [必须] 数值准确无误

**分析专业性**（40分）：
- [必须] 运用理论框架（"根据经济周期理论..."）
- [必须] 分析逻辑清晰（数据→趋势→原因→影响）
- [必须] 结合知识库内容

**结论清晰度**（30分）：
- [必须] 给出明确判断（"当前处于XX周期"）
- [必须] 提出可行建议（"建议配置XX资产"）
- [必须] 易于理解（非专业人士能看懂）

**总分≥80分：优秀** | **60-79分：良好** | **<60分：需优化**
